# Create bike network

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import numpy as np
import pandas as pd
import geopandas as gpd

import shapely.geometry as sg
import shapely.ops as so

import bgt_utils
import poly_utils

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import osmnx as ox

import folium

import settings as st
if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

## Import BGT data

In [ ]:
# Get BGT data
df_bgt_full = bgt_utils.get_bgt_data_for_bbox(st.bbox, st.bgt_layers + st.bgt_layers_bike)
df_bgt_full['naam'].value_counts()

## Select pilot areas

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)

In [ ]:
# Only keep BGT data within pilot areas
df_bgt = df_bgt_full.sjoin(df_areas, how='inner', predicate='within')  # note: only polygons fully inside area are included
df_bgt['naam_left'].value_counts()

## Import bike data from OSM

In [ ]:
# Put buffer around areas to include more lines from OSM
df_areas['buffer'] = df_areas.buffer(100)

# Retrieve the street network from selected area
polygon = df_areas.set_geometry('buffer').to_crs('EPSG:4326').unary_union
cycling_graph = ox.graph_from_polygon(polygon=polygon, network_type='bike', retain_all=True)

In [ ]:
# Fetch nodes and edges of cycling networks
gdf_nodes_cycling, gdf_edges_cycling = ox.graph_to_gdfs(
        cycling_graph,
        nodes=True, edges=True,
        node_geometry=True,
        fill_edge_geometry=True)
#gdf_edges_cycling.head()

## Select which cycling edges to include

In [ ]:
# Select bike polygons
df_bgt_bike_map = df_bgt[df_bgt['naam_left'] == 'fietspad'].to_crs("EPSG:4326").drop('index_right', axis=1)
df_bgt_bike_map['bikepath_id'] = range(0,len(df_bgt_bike_map))
df_bgt_bike_map.head(2)

In [ ]:
#gdf_edges_cycling[['highway']].reset_index(drop=True).drop_duplicates()

In [ ]:
# Select correct bike edges
gdf_edges_cycling_sel = gdf_edges_cycling[gdf_edges_cycling['highway'].isin(['cycleway', ['cycleway', 'unclassified'],  ['unclassified', 'cycleway'], 
                                                                            ['cycleway', 'service'], ['residential', 'cycleway']])]  # TODO decide on selection
gdf_edges_cycling_sel.shape

In [ ]:
# Select edges within polygons
gdf_edges_cycling_sel['unique_id'] = range(0, len(gdf_edges_cycling_sel))
gdf_edges_cycling_is = gdf_edges_cycling_sel.sjoin(df_bgt_bike_map, predicate='intersects', how='inner')
gdf_edges_cycling_is = gdf_edges_cycling_is.drop_duplicates('unique_id')  # drop duplicates/in multiple polygons
gdf_edges_cycling_is.shape

## Validate

In [ ]:
# create buffer for validation direction (left-handed)
gdf_edges_cycling_is['buffer'] = gdf_edges_cycling_is.buffer(0.0001, single_sided=True)

In [ ]:
# Set center of map
area_coords = [52.375664816888225, 4.8632280955697995]

# Set to map coordinate reference system
df_bgt_show = df_bgt.to_crs("EPSG:4326")

# Create basic map
my_map = folium.Map(
    location=area_coords,
    tiles='cartodbpositron',
    min_zoom=10, max_zoom=20, zoom_start=13,
    zoom_control=True, control_scale=True, control=False
    )

# Add sidewalks  
#geo_j = folium.GeoJson(df_bgt_show[df_bgt_show['naam_left'].isin(['voetpad', 'inrit', 'voetgangersgebied'])], style_function=lambda x: {"fillColor": "blue"}).add_to(my_map)

# Add bike paths
geo_j = folium.GeoJson(df_bgt_show[df_bgt_show['naam_left'] == 'fietspad'], style_function=lambda x: {"fillColor": "red"}).add_to(my_map)

# Add all cycling edges
geo_j = folium.GeoJson(gdf_edges_cycling['geometry'], style_function=lambda x: {"color": 'pink', "weight": 1}).add_to(my_map)

# Add buffer of cycling edges
geo_j = folium.GeoJson(gdf_edges_cycling_is['buffer'], style_function=lambda x: {"color": 'orange', "weight": 1}).add_to(my_map)

# Add selected cycling edges
geo_j = folium.GeoJson(gdf_edges_cycling_is['geometry'], style_function=lambda x: {"color": 'red', "weight": 2}).add_to(my_map)

my_map

In [ ]:
# Store map
my_map.save(cf.bike_network_map)

## Store

In [ ]:
# Prepare final bike network dataframe
gdf_edges_cycling_is['bl_id'] = range(0, len(gdf_edges_cycling_is))
gdf_final = gdf_edges_cycling_is[['bl_id', 'geometry', 'length', 'bikepath_id']].reset_index()
gdf_final = gdf_final.to_crs(st.CRS)
gdf_final.head(3)

In [ ]:
# Write bike network to file
gdf_final.reset_index().to_file(cf.output_bike_network, driver='GPKG')

In [ ]:
# Prepare bike path polygon dataframe
gdf_bikepaths = df_bgt_bike_map[['geometry', 'bikepath_id']].reset_index(drop=True)
gdf_bikepaths = gdf_bikepaths.to_crs(st.CRS)
gdf_bikepaths.head(3)

In [ ]:
# Write bike path polygons related to bike network to file
gdf_bikepaths.to_file(cf.output_bikepaths_bike_network, driver='GPKG')